# GLM

This notebook provides brief instruction describing how to run general linear model (GLM) analysis on fMRI data with Python.

## Video lectures on GLM

To learn more about GLM, please watch the lecture videos below.

- [Principles of fMRI Part 1 Module 15: The General Linear Model - Intro - YouTube](https://www.youtube.com/watch?v=GDkLQuV4he4&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 16: GLM applied to fMRI - YouTube](https://www.youtube.com/watch?v=OyLKMb9FNhg&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 17: Model Building I – conditions and contrasts - YouTube](https://www.youtube.com/watch?v=7MibM1ATai4&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 18: Model Building II – temporal basis sets - YouTube](https://www.youtube.com/watch?v=YfeMIcDWwko&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 19: Model Building III- nuisance variables - YouTube](https://www.youtube.com/watch?v=DEtwsFdFwYc&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 20: GLM Estimation - YouTube](https://www.youtube.com/watch?v=Ab-5AbJ8gAs&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 21: Noise Models- AR models - YouTube](https://www.youtube.com/watch?v=Mb9LDzvhecY&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 22: Inference- Contrasts and t-tests - YouTube](https://www.youtube.com/watch?v=NRunOo7EKD8&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 26: Multiple Comparisons - YouTube](https://www.youtube.com/watch?v=AalIM9-5-Pk&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 27: FWER Correction - YouTube](https://www.youtube.com/watch?v=MxQeEdVNihg&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 28: FDR Correction - YouTube](https://www.youtube.com/watch?v=W9ogBO4GEzA&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)
- [Principles of fMRI Part 1, Module 29: More about multiple comparisons - YouTube](https://www.youtube.com/watch?v=N7Iittt8HrU&list=PLfXA4opIOVrGHncHRxI3Qa5GeCSudwmxM)

## Setup

*For Kamitani lab members:*

Please skip the following cell if you run this notebook in Kamitani lab servers.

In [ ]:
!pip install numpy
!pip install nipy
!pip install nilearn
!pip install git+https://github.com/KamitaniLab/bdpy.git

### Download data

In [ ]:
!mkdir data

In [ ]:
# Subject 2
!curl https://ndownloader.figshare.com/files/28089525\?private_link=3bd9a1c29f19649c8c0d -o data/sub-02_task-localizer_bold_preproc_native.h5
!curl https://ndownloader.figshare.com/files/28089570\?private_link=3bd9a1c29f19649c8c0d -o data/sub-02_anatomy_t1.nii.gz
!curl https://ndownloader.figshare.com/files/28089528\?private_link=3bd9a1c29f19649c8c0d -o data/sub-02_template_native.nii.gz

In [ ]:
# Subject 3
!curl https://ndownloader.figshare.com/files/28089534\?private_link=3bd9a1c29f19649c8c0d -o data/sub-03_task-localizer_bold_preproc_native.h5
!curl https://ndownloader.figshare.com/files/28089582\?private_link=3bd9a1c29f19649c8c0d -o data/sub-03_anatomy_t1.nii.gz
!curl https://ndownloader.figshare.com/files/28089537\?private_link=3bd9a1c29f19649c8c0d -o data/sub-03_template_native.nii.gz

### Import modules

In [ ]:
import os
from itertools import product

import bdpy
from bdpy.mri import export_brain_image
import matplotlib.pyplot as plt
import numpy as np
from nipy.modalities.fmri.glm import GeneralLinearModel
from nipy.modalities.fmri.experimental_paradigm import BlockParadigm
from nipy.modalities.fmri.design_matrix import make_dmtx
import nibabel
from nilearn import plotting

## fMRI data

In this example, we run GLM analysis on fMRI data collected in the **higher visual areas localizer experiment**.
The aim of the experiment is to identify visual areas related to processing of complex visual infromation such as objects, faces, or scenes.
During the experiment, a subject was required to look at an image presented in the scanner. The image was either of them.

- Object images (IntactObject)
- Scrambled version of the object images (ScrambledObject)
- Face images (IntaceFace)
- Scrambled version of the face images (ScrambledFace)
- Scene Images (IntaceScene)
- Scrambled version of the scene images (ScrambledFace)

Example images:

<img src="https://drive.google.com/uc?export=view&id=1zyctUwtSJIWEL2xoT6ihkNMwOwgrE7mc&usp=sharing" width="200px"> <img src="https://drive.google.com/uc?export=view&id=1jR9-2uWJGHpJKN_CWucwYZ3ixT8nYbiT&usp=sharing" width="200px">

<img src="https://drive.google.com/uc?export=view&id=1iXqsH8BOuTOf2bg-VSiyP5LSql7mcO9u&usp=sharing" width="200px"> <img src="https://drive.google.com/uc?export=view&id=1xiwe2hETdm1QaLZQYxCk_fvMWwK2PyeJ&usp=sharing" width="200px">

<img src="https://drive.google.com/uc?export=view&id=1kCd9w-wzhI_88ahT1ubajfJefQiIcrap&usp=sharing" width="200px"> <img src="https://drive.google.com/uc?export=view&id=1aRW6_4DnTKAic4k4kp-sOiebdKOfJnmf&usp=sharing" width="200px">

By taking contrast between intact and scrambled images of a particular domain (e.g., IntactObject vs. ScrambledObject), we can get a brain regions related to the visual processing of the domain (e.g., lateral occipital complex for object vision).

Here is a list of typical brain regions identified with the experiment.

- LOC (lateral occipital complex; related to object recognition; Malach et al., 1995)
- FFA (fusiform face area; related to face perception; Kanwisher et al., 1997)
- PPA (parahipocampal place area; related to scene perception; Epstein & Kanwisher, 1998)

<img src="https://prosopagnosiabytmayo.files.wordpress.com/2014/11/ffa.gif" width="800px">

From: Prosopagnosia | Fusiform Face Area Functions and Impact <https://prosopagnosiabytmayo.wordpress.com/>


An experiment that identifies a brain regiton (region-of-interest; ROI) based on the function of the brain (e.g., how voxels are activated by certain stimuli) is called an *functional localizer experiment*.

### Experimet information

- TR: 3 sec
- 100 volumes/run (300 sec/run)
- 8 run

```
Blank (24 s)  Stim. (15 s)  Stim. (15 s)  Blank (15 s)                             Blank (6 s)
+----------+  +----------+  +----------+  +----------+                             +----------+
|          |  |  Intact  |  |Scrambled |  |          |                             |          |
|          |  |  Object  |  |   Face   |  |          |  ... (repated 6 times) ...  |          |
+----------+  +----------+  +----------+  +----------+                             +----------+
```

- Stimulus presentation: different images in the category were flashed in every 500 ms with blank.

### fMRI data format in this example

The fMRI data are saved as *BData*, machine-learning analysis oritented data format developed in Kamitani Lab. [bdpy](https://github.com/KamitaniLab/bdpy) is required to read the BData.

In [ ]:
bdata = bdpy.BData('data/sub-02_task-localizer_bold_preproc_native.h5')

In [ ]:
# Get fMRI data
fmri_data = bdata.select('VoxelData')
fmri_data.shape  # volumes x voxels

So the fMRI data are composed of 800 volumes and  32028 voxels.

In [ ]:
# Get labels
labels = bdata.get_label('block_type')
np.unique(labels)

The data have nine events. Three of them ('PreRest', 'InterRest', and 'PostRest') are 'rest' event in which no visual stimulus was presented. The rest event is not included in the GLM model.

In the other six events ('IntactFace', 'IntactObject', 'IntactScene', 'ScrambledFace', and 'ScrambledObject'), a visual stimulus was presented. These 'task' event should be included in the GLM model.

The labels are used to create *task regressors*, which model the signal changes caused by the experimental conditions (e.g., stimuli).

In [ ]:
# Get run numbers
runs = bdata.select('Run')
runs.shape  # samplex x 1

This is an array including run numbers, used for create *run regressors*, which explicitly model run-wise fluctuations of the fMRI signals.

## GLM setup

In [ ]:
# Config
tr = 3 # TR in sec

### Construct the design matrix

In [ ]:
# Creating run regressors
def make_runregressors(runs):
    '''Returns run regressors and labels.'''
    run_reg = np.zeros((runs.shape[0], len(np.unique(runs))))
    run_reg_labels = []
    
    for i, r in enumerate(np.unique(runs)):
        run_reg_labels.append('run-%02d' % r)
        run_reg[runs.flatten() == r, i] = 1
    
    return run_reg, run_reg_labels

In [ ]:
# Creating design matrix

ignore_event = ['PreRest', 'InterRest', 'PostRest']

condition, onset, duration = [], [], []
last_onset = 0
i = 0
tr = 3
while i < len(labels):
    lb = labels[i]
    nvol = 0
    while i < len(labels) and lb == labels[i]:
        nvol += 1
        i += 1
    cn = lb
    on = last_onset
    dr = nvol * tr
    last_onset = on + dr
    if lb in ignore_event:
        continue
    condition.append(cn)
    onset.append(on)
    duration.append(dr)

paradigm = BlockParadigm(con_id=condition, onset=onset, duration=duration)

run_regressors, run_regressors_labels = make_runregressors(runs)

n_total_vols = len(labels)
frametimes = np.linspace(0.5 * tr, n_total_vols * tr - 0.5 * tr, n_total_vols)

design = make_dmtx(frametimes, paradigm,
                   hrf_model='canonical', drift_model='cosine', hfcut=128,
                   add_regs=run_regressors,
                   add_reg_names=run_regressors_labels)

In [ ]:
# Checking the design matrix
print(design.names)
plt.plot(design.matrix[:100, 0:6])

The first six regressors stand for the experimental conditons.
The rest 45 regressors are nuisance regressors, which model signals irrelevant to the experimental conditions.

## Run GLM

Note that the fMRI data should be spatially smoothed before fitting a GLM model to make the data satisfying assumption underlying GLM.
In this example, we omit the smoothing for simplicity.

In [ ]:
# GLM
model = GeneralLinearModel(design.matrix)
model.fit(fmri_data, model='ar1')

## Visualizing results

### Visualizing contrasts

In [ ]:
# Define contrast and get statistics (z score)
contrast = [0, 1, 0, 0, -1, 0] + [0] * 45  # Intract object vs scrambled object
z_vals = model.contrast(contrast).z_score() # z score

In [ ]:
# Function converting converting z_vals to a brain image
def convert_mri_vector_to_3d(mri_vector, voxel_xyz, brain_template):
    mri_table = {}
    for i in range(mri_vector.shape[0]):
        x, y, z = voxel_xyz[0, i], voxel_xyz[1, i], voxel_xyz[2, i]
        mri_table.update({(x, y, z): mri_vector[i]})

    mri_3d = np.zeros(brain_template.shape)
    for i, j, k in product(range(brain_template.shape[0]),
                           range(brain_template.shape[1]),
                           range(brain_template.shape[2])):
        # Voxel index --> coordinates
        x, y, z = brain_template.affine[:3, :3].dot([i, j, k]) + brain_template.affine[:3, 3]
        if (x, y, z) in mri_table:
            mri_3d[i, j, k] = mri_table[(x, y, z)]

    return mri_3d

In [ ]:
voxel_xyz = np.vstack([
    bdata.get_metadata('voxel_x', where='VoxelData'),
    bdata.get_metadata('voxel_y', where='VoxelData'),
    bdata.get_metadata('voxel_z', where='VoxelData'),
])
brain_template = nibabel.load('data/sub-02_template_native.nii.gz')

z_vals_3d = convert_mri_vector_to_3d(z_vals, voxel_xyz, brain_template)

In [ ]:
plt.imshow(z_vals_3d[:, :, 23])

In [ ]:
z_vals_img = nibabel.Nifti1Image(z_vals_3d, brain_template.affine)

fig = plt.figure(figsize=(11.69, 8.27))
display = plotting.plot_anat('data/sub-02_anatomy_t1.nii.gz',
                             figure=fig, cut_coords=(-38, -78, 20))
display.add_overlay(z_vals_img, threshold=3, colorbar=True)

## Exercise tasks

**Task 1**: Visualize different contrast and see which part of the brain is activated.

In [ ]:
# Your code comes here

**Task 2**: [Optional] Try running GLM analysis on the other subject (sub-03) and visualize the results.

In [ ]:
# Your code comes here

**Task 3**: [Optional] Why some voxels that are apparently not related to the task seem to be activated? How we can avoid to get such acitvation?

Your answer comes here: